# Prédiction de notes de films Allociné en fonction d'avis


In [ ]:
import os
import sys

import numpy as np

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

Nous utiliserons le jeu de données `allocine`

In [18]:
from datasets import load_dataset
dataset = load_dataset("allocine")

c:\Users\FLOWUP\Desktop\FILES\PROGRAMMATION\easyTrainer\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\FLOWUP\Desktop\FILES\PROGRAMMATION\easyTrainer\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FLOWUP\.cache\huggingface\hub\datasets--allocine. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

In [34]:
train = dataset["train"]
test = dataset["test"]
val = dataset["validation"]

In [37]:
print(train["review"][0])
print(train["label"][0])

Si vous cherchez du cinéma abrutissant à tous les étages,n'ayant aucune peur du cliché en castagnettes et moralement douteux,"From Paris with love" est fait pour vous.Toutes les productions Besson,via sa filière EuropaCorp ont de quoi faire naître la moquerie.Paris y est encore une fois montrée comme une capitale exotique,mais attention si l'on se dirige vers la banlieue,on y trouve tout plein d'intégristes musulmans prêts à faire sauter le caisson d'une ambassadrice américaine.Nauséeux.Alors on se dit qu'on va au moins pouvoir apprécier la déconnade d'un classique buddy-movie avec le jeune agent aux dents longues obligé de faire équipe avec un vieux lou complètement timbré.Mais d'un côté,on a un Jonathan Rhys-meyers fayot au possible,et de l'autre un John Travolta en total délire narcissico-badass,crâne rasé et bouc proéminent à l'appui.Sinon,il n'y a aucun scénario.Seulement,des poursuites débiles sur l'autoroute,Travolta qui étale 10 mecs à l'arme blanche en 8 mouvements(!!)ou laiss

Commençons par nettoyer les avis de tous les mots qui peuvent être "parasites".

Importons `TextualPreparator` et définissons le nettoyage que nous voulons appliquer

In [31]:
from easyTrainer.data.text import TextualPreparator

In [38]:
TP = TextualPreparator(
    to_lower=1,
    drop_stopwords=2,
    drop_digits=3,
    drop_special_characters=4,
    drop_accents=5,
    lemmatize=6,
    drop_words_less_than_N_letters=(7, 4, False),
    drop_big_spaces=8
)

print(TP)

TextualPreparator with transformations: txt_to_lower@1, drop_stopwords@2, drop_digits@3, drop_special_characters@4, drop_accents@5, lemmatize@6, drop_words_less_than_N_letters@7, drop_big_spaces@8


Nous pouvons lancer la préparation. En sortie, nous voulons des données encodées, c'est pourquoi nous utilisons `encoder_name_to_fit="tfidf"`.

In [39]:
train_prep = TP.prepare(train["review"], encoder_name_to_fit="tfidf")

Cela nous retourne nos données nettoyées, encodées, et notre encoder `TfidfVectorizer`. Nous pourrons ensuite l'appliquer sur les données de test et de validation

In [40]:
train_prep

{'data': ['cherchez cinemer abrutissant etage avoir peur cliche castagnette moralement douteux from pari with love vous tout production besson filiere europacorp faire naitr moquerie pari montree capital exotique mais attention dirige banlieue trouver integriste musulman pret faire sauter caisson ambassadric americaine nauseeux alors pouvoir apprecier deconnad classique buddy movie jeune agent dent long oblige faire equipe vieux completement timbre mais cote jonathan rhys meyer fayot possible autre john travolta total delir narcissico badas crane rase bouc proeminent appui sinon scenario seulement poursuite debile autoroute travolta etal arme blanc mouvement laisse associer faire demolir tronche scruter jumeau plaisir coupable tellement être henaurme être daube ligne transporteur taken banlieue',
  'trash trash trash horreur imaginer vous premiere minute oranger mecaniqu dilatees minute bande pourrave revisitee korine sauc anneer dandy punk kubrick laisse place papy lubrique deguiser p

In [41]:
X_train = train_prep["encoded_data"]
y_train = train["label"]
vectorizer = train_prep["vectorizer"]

Préparons nos données de test et de validation en appliquant notre encodeur entrainé avec `custom_encoder_fit=vectorizer`.

In [42]:
test_prep = TP.prepare(test["review"], custom_encoder_fit=vectorizer)
val_prep = TP.prepare(val["review"], custom_encoder_fit=vectorizer)

X_test = test_prep["encoded_data"]
y_test = test["label"]

X_val = val_prep["encoded_data"]
y_val = val["label"]

Entrainons un modèle

In [43]:
from sklearn.ensemble import RandomForestClassifier

from easyTrainer.models.classifier import SklearnClassifierModel

In [44]:
SCM = SklearnClassifierModel(
    model=RandomForestClassifier(n_estimators=100, random_state=42),
)

In [45]:
SCM.fit(
    X = X_train,
    y = y_train
)

KeyboardInterrupt: 

In [ ]:
SCM.test(
    X= X_test, 
    y= y_test
)

,1,2,3,4,5,accuracy,macro avg,weighted avg
precision,0.416667,0.166667,0.387640,0.328244,0.692251,0.628319,0.398294,0.541423
recall,0.044643,0.010204,0.341584,0.140065,0.970287,0.628319,0.301357,0.628319
f1-score,0.080645,0.019231,0.363158,0.196347,0.808020,0.628319,0.293480,0.550549
support,112.000000,98.000000,202.000000,307.000000,976.000000,0.628319,1695.000000,1695.000000


In [ ]:
SCM.validation(
    X = X_val, 
    y = y_val,
    fallback_class=3,
    validation_thresholds=np.arange(0.5, 1.0, 0.05)
)

{'thresholds': [np.float64(0.5),
  np.float64(0.55),
  np.float64(0.6000000000000001),
  np.float64(0.6500000000000001),
  np.float64(0.7000000000000002),
  np.float64(0.7500000000000002),
  np.float64(0.8000000000000003),
  np.float64(0.8500000000000003),
  np.float64(0.9000000000000004),
  np.float64(0.9500000000000004)],
 'accuracy': [0.6064896755162242,
  0.5929203539823009,
  0.5710914454277286,
  0.5457227138643068,
  0.5162241887905604,
  0.48436578171091443,
  0.4471976401179941,
  0.3793510324483776,
  0.30383480825958703,
  0.20648967551622419],
 'f1_score_weighted': [0.5346197499752163,
  0.5314036920900957,
  0.521274230177564,
  0.5073704745616436,
  0.4906512446366674,
  0.46922234525817974,
  0.44155649569080574,
  0.38333143642688755,
  0.3062311512329151,
  0.17952839201504303],
 'f1_score_macro': [0.2527913272891002,
  0.24956587869382907,
  0.24028375795282103,
  0.23259884603067102,
  0.2233196416759895,
  0.21309467112240607,
  0.20123361634020598,
  0.177102954119